In [1]:
import pandas as pd
import numpy as np
import re
import glob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

import datetime
import nltk
import time

from shared_lib import utils, vocabulary
from shared_lib import ngram_lm
from shared_lib import ngram_utils
from shared_lib import simple_trigram

%matplotlib inline



In [2]:
boston_bomb_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/boston_comments_2013.03-2013.05.txt', lines=True)
boston_series_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/boston_comments_2013.10-2013.11.txt', lines=True)
colorado_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/colorado_comments_2017.06-2017.09.txt', lines=True)
florida_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/florida_comments_2017.06-2017.09.txt', lines=True)
houston_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/houston_comments_2017.06-2017.09.txt', lines=True)
miami_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/miami_comments_2017.06-2017.09.txt', lines=True)
nyc_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/nyc_comments_2012.08-2012.12.txt', lines=True)
puerto_rico_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/puerto_rico_comments_2017.06-2017.09.txt', lines=True)
vegas_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/vegas_comments_2017.06-2017.09.txt', lines=True)

In [36]:
n_samples = 2000
n_features = 1000
n_components = 30
n_top_words = 20

In [37]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [38]:
#np.sum(pd.read_csv("/Users/krista/Desktop/w266-project-master/data/reddit/vegas_comments_2017.06-2017.09.txt", delimiter=";")["favorites"] < 1)

In [39]:
pattern = "http"

In [40]:
dirs = glob.glob("/Users/krista/Desktop/w266-project-master/data/reddit/vegas_comments_2017.06-2017.09.txt")
sentences = []
for dir_ in dirs:
    try:
        df = pd.read_json(dir_, lines=True)
#         df = df[~df.text.str.contains(pattern)]
        new_sentences = list(df['body'].values)
        for sentence in new_sentences:
#             regex = re.compile('[^a-zA-Z]')
#             sentence = regex.sub(sentence, regex)
            sentence = re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
            sentence = re.sub(" \d+", '', sentence)
            sentence = re.sub(r'\w*\d\w*', '', sentence)
        
            sentences.append(sentence)
    except Exception as e:
        print e

In [59]:
vegastxt='/Users/krista/Desktop/w266-project-master/data/reddit/vegas_comments_2017.06-2017.09.txt'

def word_count(filename, word):
    with open(filename, 'r') as f:
        return f.read().count(word)
    
print word_count(vegastxt, 'mandalay'),  'mandalay'
print word_count(vegastxt, 'shooting'), 'shooting'
print word_count(vegastxt, 'mandalay'),  'mandalay bay'
print word_count(vegastxt, 'shooting'), 'pray'
print word_count(vegastxt, 'dead'), 'dead'
print word_count(vegastxt, 'Paddock'), 'Paddock'

7 mandalay
49 shooting
7 mandalay bay
49 pray
55 dead


In [58]:
print len(set(w.lower() for w in open(vegastxt).read().split()))

75655


In [41]:
print len(sentences)

17838


In [42]:
sentences[:30]

[u"Don't these street picture people make tons of money? ",
 u"hmm... some mixed reviews. I'm going to be in that area tomorrow, I'll stop by and check them out.\n\nThanks for the recommendation.\n",
 u'[deleted]',
 u'My SO and I are planning on coming with our three year old. ',
 u"Also call Richard Carrillo's office or email him.  I don't want to get banned for posting his email but you can find it by searching his name.\n\nHere is what I wrote to him,\n\n&gt;Assemblyman Carrillo,\n\n\n&gt;I would advise you to rescind this amendment. \xa0Ridesharing in Nevada has prevented numerous incidents of impaired driving in Nevada. \xa0This service is saving countless lives and reducing economic burden to those involved in impaired driving accidents.\n\n\n&gt;We, the citizens of Nevada, will not forget this if you get ridesharing taken away from us.\n\n\n&gt;Thank you for considering the safety of the citizens of Nevada.\n\nHe is the representative of District. ",
 u"Idk, I'm all for Uber dri

In [43]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=10,
                                   max_features=10**5,
                                   stop_words='english',
                                   strip_accents="ascii"
                                  )
tfidf = tfidf_vectorizer.fit_transform(sentences)

In [44]:
tfidf_vectorizer.vocabulary_.keys()[:50]

[u'hendertucky',
 u'hanging',
 u'increase',
 u'bringing',
 u'wednesday',
 u'thunder',
 u'kids',
 u'golden',
 u'replace',
 u'music',
 u'strike',
 u'paperwork',
 u'circumstances',
 u'locked',
 u'harrahs',
 u'example',
 u'want',
 u'absolute',
 u'travel',
 u'feature',
 u'wrong',
 u'excited',
 u'types',
 u'fit',
 u'fix',
 u'easier',
 u'effects',
 u'schools',
 u'volcano',
 u'development',
 u'directly',
 u'series',
 u'drove',
 u'rd',
 u'foundation',
 u'moment',
 u'caught',
 u'speeds',
 u'renovated',
 u'service',
 u'needed',
 u'master',
 u'rewards',
 u'showed',
 u'tree',
 u'shower',
 u'feeling',
 u'dozen',
 u'recommended',
 u'doors']

In [45]:
print tfidf.shape

(17838, 3498)


In [46]:
tfidf = tfidf.todense()
tfidf = np.unique(tfidf, axis=0)

In [47]:
print tfidf.shape
from scipy import sparse
tfidf = sparse.csr_matrix(tfidf)

(15585, 3498)


In [48]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [49]:
nmf = NMF(n_components=n_components, random_state=1,
          max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words=10)

Topic #0: going want day way work lot money area night pay better town gt nice best need probably make year looking
Topic #1: com www https http amp reddit imgur message youtube watch html jpg comments np compose facebook news link gallery en
Topic #2: thanks info appreciate awesome man reply help definitely advice heads response wow suggestions ok link helpful cool sharing input recommendation
Topic #3: vegas las north blvd city live lake best la henderson living welcome groupon areas parts downtown area favorite lived road
Topic #4: good luck idea sounds food stuff thing point damn deal steak thought spot selection hear prices buffet pizza start sidebar
Topic #5: like looks sounds feel doesn kinda shit stuff live things sound feels fun dress hope little doing looked thought said
Topic #6: don know let need didn care use want doing come mind forget asking understand help sorry friends dont comment works
Topic #7: just say read saw moved mean don trying maybe wanted walk wondering post

In [50]:
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

In [51]:
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)



Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: car high end away bit hard way little minutes especially use getting leave experience half long just home large don
Topic #1: com https www amp http reddit imgur google facebook message news vegas watch comments jpg link youtube html org county
Topic #2: thanks look info help ok appreciate ah reply ll link interesting information sidebar sweet app wow post definitely suggestions site
Topic #3: vegas las north city downtown henderson live area lake charleston living la blvd east trip areas boulder valley stratosphere town
Topic #4: good luck idea like food beer steak buffet chicken island decent cheap prices tacos sounds selection recommend ellis museum station
Topic #5: like looks sounds feel kinda things op fuck look sound doing movie kids vacation kind dick feels doesn hookers joke
Topic #6: know don let come use need care asking friends talking happen stupid vote meet comment fact answer help idea business
Top

In [29]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

Extracting tf features for LDA...


In [30]:
tf = tf_vectorizer.fit_transform(sentences)


In [31]:
lda = LatentDirichletAllocation(max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [32]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [33]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0: people don got just like did shit think said pay weekend work month line getting job life man long real
Topic #1: strip like don just vegas want good need night free people best place places nice food going bar know ll
Topic #2: isn right ll yes money buy don try know like day kind people long better make really different going way
Topic #3: com vegas las https www http room lol mgm check red view saw hotel property rock north location resort casino
Topic #4: ve time area great lot love drive experience car just place vegas good bad water house year hours town like
Topic #5: doesn tip club let service uber just use money way fuck safe time pay tips state won make lyft nevada
Topic #6: deleted gt new old does care help hit guy sell months phone days coming dude guys search closed girls imagine
Topic #7: know yeah worth thank week think didn ago mean went read couple won looking awesome list weeks called guest question
Topic #8: good just sure come pretty point say actually coo